# Assignment 6
#### Problem 2

Jérémie Engler and Laure Toullier

07.12.2023

Aim : analysis of a cantilever beam-column element subjected to a tip force

In [248]:
#imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math

np.set_printoptions(precision=5, suppress=True, linewidth=150)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

In [249]:
#Data

a = 200 #mm
b = 200 #mm
L = 2000 #mm (uniform XS)
E = 200000 #MPa (elastic material)
n = 5 #number of intergration point
nf = 10 #number of fibers in the XS
stress_fibers = 1 #MPa


Step 1 : Implement and validate a displacement based fiber beam-column element

In [250]:
#computation of tangent section stiffness matrix and section resisting force

def ks_fs(nf):

    yk=np.zeros((nf))    #array for each the coordinate of each centroïd of each fiber
    lk=np.zeros((nf,2))
    ks=np.zeros((2,2))   #tangent section stiffness matrix
    fs=np.zeros((2,1))   #section resisting force

    for i in range(nf):
        yk[i]=-b/2+b/(2*nf)+b/nf*((i+1)-1) #centroïd of each fiber
        lki=np.array([[1,yk[i]]])
        lk[i]=lki
        ks+=np.transpose(lki)@lki*E*(a*b/nf)
        fs+=np.transpose(lki)*stress_fibers*(a*b/nf)

    return ks, fs

#note: as the XS is constant, ks and fs are constant along the cantilever

The numerical integration used to obtain a numerical estimate of the integral is Gauss-Lobato, with 5 integration points. The conditions that satisfy all error partial derivatives for 5 integration points are:

    w1=w5=1/10 and r1=1, r5=-1

    w2=w4=49/90 and r2=-sqrt(21)/7, r4=sqrt(21)/7 
    
    w3=32/45 and r3=0

In [251]:
#computation of the stiffness matrix

r=np.array([-1,-np.sqrt(21)/7, 0, np.sqrt(21)/7,1])
w=np.array([0.1,49/90,32/45,49/90,0.1])

def Ke(L,w,r,k_s,f_s):
    
    Ke=np.zeros((6,6))
    Qfv=np.zeros((6,1))

    for i in range(n) :
        xi=L/2*r[i]+L/2      #we integrate using a variable transformation from r to x to normalize the system: x=L/2*r+L/2 and dx=L/2*dr 
        
        B=np.array([[-1/L, 0, 0, 1/L, 0,0],
                [0, (12*xi/(L**3))-6/(L**2), 6*xi/(L**2)-4/L, 0, -12*xi/(L**3)+6/(L**2) , 6*xi/(L**2)-2/L]])
        
        kb=(B.T@k_s)@B
        Ke+=L/2*w[i]*kb

        qb=B.T@f_s
        Qfv+=L/2*qb*w[i]

    return Ke, Qfv




In [252]:
k_s,f_s=ks_fs(nf)

K_e,Q_e=Ke(L,w,r,k_s,f_s)


In [253]:
print(f"k_s={k_s}")
print(f"f_s={f_s}")
print(f"The element resisting force vector is ")
print(Q_e)
print("The tangent element stiffness matrix of a displacement based fiber BC elemnt is:")
print(K_e)

k_s=[[8.00e+09 0.00e+00]
 [0.00e+00 2.64e+13]]
f_s=[[40000.]
 [    0.]]
The element resisting force vector is 
[[-40000.]
 [     0.]
 [     0.]
 [ 40000.]
 [     0.]
 [     0.]]
The tangent element stiffness matrix of a displacement based fiber BC elemnt is:
[[ 4.00e+06  0.00e+00  0.00e+00 -4.00e+06  0.00e+00  0.00e+00]
 [ 0.00e+00  3.96e+04  3.96e+07  0.00e+00 -3.96e+04  3.96e+07]
 [ 0.00e+00  3.96e+07  5.28e+10  0.00e+00 -3.96e+07  2.64e+10]
 [-4.00e+06  0.00e+00  0.00e+00  4.00e+06  0.00e+00  0.00e+00]
 [ 0.00e+00 -3.96e+04 -3.96e+07  0.00e+00  3.96e+04 -3.96e+07]
 [ 0.00e+00  3.96e+07  2.64e+10  0.00e+00 -3.96e+07  5.28e+10]]


In [254]:
#classical stiffness matrix (from week 7, slide 19)

def stiffness_elastic_beam (E,a,b,L):

    K=np.zeros((6,6))

    I=a*b**3/12
    A=a*b
    
    K[0,0]=E*A/L
    K[0,3]=-E*A/L
    K[3,0]=-E*A/L
    K[3,3]=E*A/L

    K[1,1]=K[4,4]=E*I*12/L**3
    K[1,2]=K[2,1]=E*I*6/L**2
    K[2,2]=K[5,5]=E*I*4/L
    K[5,4]=K[4,5]=-6*E*I/L**2

    K[1,4]=K[4,1]=-E*I*12/L**3
    K[1,5]=K[5,1]=E*I*6/L**2
    K[2,4]=K[4,2]=-E*I*6/L**2
    K[2,5]=K[5,2]=E*I*2/L

    return K

K_classic=stiffness_elastic_beam (E,a,b,L)
print(f"K_classic={K_classic}")


K_classic=[[ 4.00000e+06  0.00000e+00  0.00000e+00 -4.00000e+06  0.00000e+00  0.00000e+00]
 [ 0.00000e+00  4.00000e+04  4.00000e+07  0.00000e+00 -4.00000e+04  4.00000e+07]
 [ 0.00000e+00  4.00000e+07  5.33333e+10  0.00000e+00 -4.00000e+07  2.66667e+10]
 [-4.00000e+06  0.00000e+00  0.00000e+00  4.00000e+06  0.00000e+00  0.00000e+00]
 [ 0.00000e+00 -4.00000e+04 -4.00000e+07  0.00000e+00  4.00000e+04 -4.00000e+07]
 [ 0.00000e+00  4.00000e+07  2.66667e+10  0.00000e+00 -4.00000e+07  5.33333e+10]]


In [255]:
#computation of the error

def error(nf):
    k_s,f_s=ks_fs(nf)
    K_e,Q=Ke(L,w,r,k_s,f_s)
    
    err=np.linalg.norm(K_e-K_classic)/np.linalg.norm(K_classic)*100
    return err

def error_min(err_accept):
    n_iter=20
    n=1
    while error(n)>err_accept and n<n_iter:
        n+=1
    return n


In [256]:
err=error(nf)
print(f"error={err}")
min_num_fibers=error_min(2)
err_min=error(min_num_fibers)
print(f"Minimum number of fibers for error less than 2%={min_num_fibers}")
print(f"For a minimum number of fiber less than 2%, the error is: {err_min}")

error=0.9999999955000101
Minimum number of fibers for error less than 2%=8
For a minimum number of fiber less than 2%, the error is: 1.5624999929687748
